In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Le but de ce notebook est de <b>répertorier</b> différentes fonctions ou méthodes qui vont permettre d'ajuster les paramètres de modèles ou de stratégies de nettoyage/preprocess pour avoir la MAE la plus minime possible ou tout simplement avoir des données exploitables <br>
Source de la majorité des fonctions : <a> https://www.kaggle.com/learn </a>

# 1. Fonctions pour affiner les modèles

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

## RandomForest (Regressor)

### Cross Validation 

Parametrage des n_estimators avec un cv donné

In [ ]:
def get_score_RF(n_estimators, cv=5):
    """Return the average MAE of random forest model.
    
    Keyword argument:
    n_estimators -- the number of trees in the forest
    cv --- number of validation, default 5 ( équivalent répartition du train_test_split à 0.8 0.2 ) 
    """
    pipeline = Pipeline(steps=[('imputer',SimpleImputer()),('model',RandomForestRegressor(n_estimators = n_estimators, random_state=0))])
    scores = -1 * cross_val_score(pipeline, X, y,
                              cv=cv,
                              scoring='neg_mean_absolute_error')
    return scores.mean()

### Classique MAE pour RF

pas de cv, mae directe et brute

In [6]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model= RandomForestRegressor(n_estimators=100, reandom_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## XG-Boost (Regressor)

Calcul de RMSLE ( pas de pipeline possible (?) puisqu'on l'on utilise la méthode pandas factorize pour Ordinal encode les categoricals ) <br>
Donc version à revoir pour automatiser ce problème ?

In [ ]:
def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


Parametrage des n_estimators

In [2]:
def get_score_XGBoost(n_estimator):
    model = XGBRegressor(n_estimators=n_estimator, random_state=0)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(preds, y_valid)

Parametrage du learning_rate

In [ ]:
def get_score_XGB_lr(learning_rate):
    model = XGBRegressor(n_estimators = 1000, learning_rate= learning_rate, random_state=0)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(preds, y_valid)

## K-means

Chose bête : penser à standardiser les colonnes des features qui vont être utilisées pour du clustering avant toute chose ? 

# 2 Fonctions de featuring

## 2.1 Mutual Information

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [5]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

# 3 Fonctions de preprocess 

## 3.1 Categorical Data 

Ces fonctions sont à lier avec une fonction de scoring pour déterminer quelle est la meilleure approche pour gérer ces différentes données

Fonction qui pour un set de données d'entrainement donné retourne les colonnnes catégoricielles avec leur cardinal si définit comme Vrai( pour vraiment être flemmard ) sinon renvoi 

In [9]:
def get_obj_cols(X_train, card=False):
        return [col for col in X_train.columns if X_train.col.dtype == 'object']

Fonction qui pour une liste de colonnes "catégoricielles"  et un cardinal minimum donné , retourne les colonnes à One Hot ou à Ordinal

In [10]:
def get_obj_cols_strategy(obj_cols, card):
    ordinal_cols = []
    one_hot_cols = []
    for col in obj_cols:
        col_card = X_train.col.nunique()
        if col_card >= card:
            ordinal_cols.append(col)
        else:
            one_hot_cols.append(col)
    return ordinal_cols, one_hot_cols

pour les colonnes à OH , retourne un X_train , X_valid qui ont été One Hot

In [12]:
def col_one_hot_encode(low_cardinality_cols):
    # Apply one-hot encoder to each column with categorical data
    # On pourra modifier le handle_unknown ou différents paramètres de OneHotEncoder
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
    OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = X_train.index
    OH_cols_valid.index = X_valid.index

    # Remove categorical columns (will replace with one-hot encoding)
    num_X_train = X_train.drop(object_cols, axis=1)
    num_X_valid = X_valid.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical features
    OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
    OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

    # Ensure all columns have string type
    OH_X_train.columns = OH_X_train.columns.astype(str)
    OH_X_valid.columns = OH_X_valid.columns.astype(str)
    return OH_X_train, OH_X_valid 

Meme principe pour un ordinal 

In [13]:
def col_ord_encode(high_cardinality_cols):
    ordinal_encoder = OrdinalEncoder()
    ord_X_train[high_cardinality_cols]=ordinal_encoder.fit_transform(X_train[high_cardinality_cols])
    ord_X_valid[high_cardinality_cols]=ordinal_encoder.transform(X_train[high_cardinality_cols])
    return ord_X_train, ord_X_valid

Si des catégories sont présentes sur le set de validation mais pas sur celui d'entrainement, <br>
En les "encodant" on peut avoir des soucis .... <br>
La fonction suivante permet de drop celles-ci si c'est la stratégie adoptée pour les gérer 


In [14]:
def drop_categ_cols_withpb(obj_cols):
    # Columns that can be safely ordinal encoded
    good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]      
    # Problematic columns that will be dropped from the dataset
    bad_label_cols = list(set(object_cols)-set(good_label_cols))
    
    label_X_train = X_train.drop(bad_label_cols, axis=1)
    label_X_valid = X_valid.drop(bad_label_cols, axis=1)
    return label_X_train, label_X_valid

On peut aussi utiliser des "fonctions" pandas : 

In [ ]:
# OH encode équivalent, voir doc pour arguments
pd.get_dummies(df.col)

In [ ]:
# Ordinal encode 
df["categorical_col"].factorize()

# 4 Fonctions de Nettoygae

### Erreurs de Typos

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

A condition de ne pas perdre de l'information sensible, on peut les mettres toutes en minuscule et enlever les espaces inutiles

In [ ]:
# convert to lower case
df.col = df.col.str.lower()
# remove trailing white spaces
df.col = df.col.str.strip()

In [ ]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

### Gérer les dates

In [ ]:
import datetime

In [ ]:
df['date_parsed'] = pd.to_datetime(df.date, format="%m/%d/%y") # pour un format string de mm/dd/yy , à modifier selon le dataset

Le format des entrées de dates dans une meme colonne peut etre inconsistant, on peut donc penser à vérifier les différentes longueurs de chaines de caractères présentes dans le dataset pour les repérer et écrire une fonction pour les corriger. <br>

### Mise à l'echelle (Scaling)

Changer l'echelle, Mesurer la distance entre chaque points de manière "cohérente"  , utile pour du SVM ou KNN

In [ ]:
from mlxtend.preprocessing import minmax_scaling

In [ ]:
# min-max scale the data between 0 and 1
scaled_data = minmax_scaling(original_data, columns=[0])

Pour du k-means on peut calculer : (x- x.moyenne(axis=0)) / x.ecart-type(axis=0)

In [ ]:
# X_sclaed est X comprenant les features utilisées pour le clustering
X_scaled = (X_scaled - X_scaled.means(axis=0)) / X_scaled.std(axis=0)

### Normalisation 

Changer la forme de la distribution de ses données pour l'exploiter via du Gaussian Naive Bayes ou LDA ( Linear Discriminant Analysis) 

In [ ]:
from scipy import stats

In [ ]:
normalized_data = stats.boxcox(original_data)

### Données nulles 

In [5]:
from sklearn.impute import SimpleImputer

ou le classique pandas : .fillna()

# Annexe 

##  Déterminer l'encodage d'un fichier pour le lire 

In [15]:
import charset_normalizer

In [ ]:
with open("path/your_file.csv", 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read())
print(result)

In [ ]:
df = pd.read_csv("path/your_file.csv", encoding='enter_the_encoding_result_here')